The aim of this script is to simulate the motion a chaser satellite in close proximity to a target satellite. This uses the Clohessy Wiltshire equations to draw the chaser satellite in the reference frame of the target satellite. This particular sumulation attempts to simulate the motion of the satellite given inital condistions and then provides the means to apply an extended burn at a across a chosen time period and fuel capacity. 


This script works on making the fuel compoent of the siluation more accurate.

In [2]:
import numpy as np
import plotly.graph_objects as go

# Gravitational parameter (mu) and mean motion (n)
mu = 398600.4418  # km^3/s^2 (gravitational parameter of Earth)
sma = 7000  # km (semi-major axis of the target's orbit)
n = np.sqrt(mu / sma**3)  # mean motion of the target

# Initial conditions in the target's orbit reference frame
r0_chaser = np.array([10.0, 9.0, -30.0])  # Initial relative position of chaser, km
v0_chaser = np.array([3, 5, 0.1])  # Initial relative velocity of chaser, km/s

# Thrust parameters
thrust_start_time = 2000  # Time delay before thrust starts in seconds
thrust_duration = 60    # Thrust duration in seconds
thrust_force = np.array([-1, 0, 0])  # Thrust force in N
specific_impulse = 3000  # Specific impulse of the propulsion system, seconds
fuel_mass_inital = 1000  # Initial mass of the fuel
spacecraft_mass = 4000 # Initial mass of the fuel, kg
total_mass_initial = spacecraft_mass + fuel_mass_inital  # Initial total mass of the spacecraft

# Time parameters
t_final = 5000  # Total simulation time, seconds
dt = 1  # Time step, seconds

# Time array
t = np.arange(0, t_final, dt)

# Position and velocity arrays
r_chaser = np.zeros((len(t), 3))
v_chaser = np.zeros((len(t), 3))

# Set initial conditions
r_chaser[0] = r0_chaser
v_chaser[0] = v0_chaser

# Clohessy-Wiltshire equations for relative motion
def clohessy_wiltshire(r, v, t, n):
    x, y, z = r
    x_dot, y_dot, z_dot = v
    
    x_ddot = 3 * n**2 * x + 2 * n * y_dot
    y_ddot = -2 * n * x_dot
    z_ddot = -n**2 * z
    
    return np.array([x_dot, y_dot, z_dot]), np.array([x_ddot, y_ddot, z_ddot])

# Simulation loop

current_mass = total_mass_initial

for i in range(1, len(t)):
    # Get current state
    r = r_chaser[i-1]
    v = v_chaser[i-1]
    
    # Find accelerations using Clohessy-Wiltshire equations
    v, a = clohessy_wiltshire(r, v, t[i], n)

    print(v,a)
    
    # Calculate thrust acceleration using rocket equation
    if thrust_start_time <= t[i] <= (thrust_start_time + thrust_duration) and (current_mass - spacecraft_mass) > 0:
        exhaust_velocity = specific_impulse * 9.81  # Exhaust velocity in m/s
        thrust_acceleration = thrust_force / current_mass  # Thrust acceleration in m/s^2
        fuel_consumed = np.linalg.norm(thrust_force) * dt / exhaust_velocity  # Fuel consumed in kg
        current_mass -= fuel_consumed  # Decrease current mass
        thrust_acceleration = thrust_acceleration * 1e-3  # Convert thrust acceleration to km/s^2
    else:
        thrust_acceleration = np.array([0.0, 0.0, 0.0])  # No thrust
    
    # Total acceleration
    total_acceleration = a + thrust_acceleration
    
    # Update position and velocity
    r_chaser[i] = r + v * dt
    v_chaser[i] = v + total_acceleration * dt

# Plotting with Plotly for interactive visualization
r_chaser_scaled = r_chaser / sma

# Create traces for plotting
trace = go.Scatter3d(
    x=r_chaser_scaled[:, 0],
    y=r_chaser_scaled[:, 1],
    z=r_chaser_scaled[:, 2],
    mode='markers',
    name='Chaser Satellite',
    marker=dict(
        color='blue',
        size=2
    )
)

initial_trace = go.Scatter3d(
    x=[r_chaser_scaled[0, 0]],
    y=[r_chaser_scaled[0, 1]],
    z=[r_chaser_scaled[0, 2]],
    mode='markers',
    name='Initial Position',
    marker=dict(
        color='black',
        size=5
    )
)

target_trace = go.Scatter3d(
    x=[0],
    y=[0],
    z=[0],
    mode='markers',
    name='Target Satellite',
    marker=dict(
        color='red',
        size=5
    )
)

thrust_start_trace = go.Scatter3d(
    x=[r_chaser_scaled[thrust_start_time, 0]],
    y=[r_chaser_scaled[thrust_start_time, 1]],
    z=[r_chaser_scaled[thrust_start_time, 2]],
    mode='markers',
    name='Thrust Start',
    marker=dict(
        color='green',
        size=5
    )
)

# Create the layout
layout = go.Layout(
    title='Relative Motion of the Chaser Satellite',
    scene=dict(
        xaxis=dict(title='X/a [km]'),
        yaxis=dict(title='Y/a [km]'),
        zaxis=dict(title='Z/a [km]')
    )
)

# Create the figure and add the traces
fig = go.Figure(data=[trace, target_trace, thrust_start_trace, initial_trace], layout=layout)

# Show the figure
fig.show()


[3.  5.  0.1] [ 1.08149391e-02 -6.46804568e-03  3.48630124e-05]
[3.01081494 4.99353195 0.10003486] [ 1.08114528e-02 -6.49136285e-03  3.47468024e-05]
[3.02162639 4.98704059 0.10006961] [ 1.08079540e-02 -6.51467251e-03  3.46305518e-05]
[3.03243435 4.98052592 0.10010424] [ 1.08044425e-02 -6.53797462e-03  3.45142609e-05]
[3.04323879 4.97398794 0.10013875] [ 1.08009185e-02 -6.56126916e-03  3.43979297e-05]
[3.05403971 4.96742668 0.10017315] [ 1.07973820e-02 -6.58455611e-03  3.42815584e-05]
[3.06483709 4.96084212 0.10020743] [ 1.07938329e-02 -6.60783543e-03  3.41651471e-05]
[3.07563092 4.95423428 0.1002416 ] [ 1.07902712e-02 -6.63110710e-03  3.40486960e-05]
[3.08642119 4.94760318 0.10027565] [ 1.07866971e-02 -6.65437109e-03  3.39322052e-05]
[3.09720789 4.94094881 0.10030958] [ 1.07831103e-02 -6.67762737e-03  3.38156749e-05]
[3.107991   4.93427118 0.1003434 ] [ 1.07795111e-02 -6.70087592e-03  3.36991051e-05]
[3.11877051 4.9275703  0.10037709] [ 1.07758993e-02 -6.72411671e-03  3.35824960e-05]
[